## Hidden Markov model

In [1]:
import pandas as pd
import numpy as np
from hmmlearn import hmm
from itertools import product
from scipy.stats import mode

In [2]:
df = pd.read_csv("data/szczecin_sep_2009.txt", sep=",", header=None)
df = df.iloc[:, [1, 3, 4, 7, -1]]
df.columns = ["temperatura", "wilgotnosc", "cisnienie", "predkosc wiatru", "condition"]

dictionary = {'Calm':0, '-9999.0':0}
df = df.replace({"predkosc wiatru": dictionary})

np.unique(df["condition"])
dictionary = {'Light Rain Showers':'Light Rain',
              'Light Thunderstorms and Rain':"Rain",
              'Mist':'Light Rain',
              'Mostly Cloudy':'Cloudy',
              'Partly Cloudy':'Cloudy',
              'Rain Showers':'Rain',
              'Scattered Clouds':'Cloudy',
              'Shallow Fog':'Fog',
              'Thunderstorms and Rain':'Rain',
              'Unknown':'Fog'}
df = df.replace({"condition": dictionary})

condition_dictionary = {'Clear':0, 'Cloudy':1, 'Fog':2, 'Light Rain':3, 'Rain':4}
df = df.replace({"condition": condition_dictionary})
df

,temperatura,wilgotnosc,cisnienie,predkosc wiatru,condition
0,14.0,77,1018,8.1,0
1,14.0,77,1018,8.1,0
2,14.0,77,1018,9.2,0
3,14.0,77,1017,9.2,0
4,14.0,77,1017,9.2,0
...,...,...,...,...,...
1433,9.0,76,1012,4.6,0
1434,9.0,76,1012,4.6,0
1435,9.0,76,1011,4.6,0
1436,9.0,76,1011,5.8,0


In [3]:
import random 

def train_test_split(df, T):
    sequences = []
    n_sequences = df.shape[0] // T
    X_train = pd.DataFrame()
    X_test = pd.DataFrame()
    for i in range(n_sequences):
        split_idx_begin = i * T
        split_idx_end = (i+1) * T
        sequences.append(df.iloc[split_idx_begin:split_idx_end, :])
    train_idx = random.sample(range(n_sequences), k = int(n_sequences * 0.7))
    test_idx = np.setdiff1d(np.arange(n_sequences), train_idx)
    for i in train_idx:
        X_train = X_train.append(sequences[i], ignore_index = True)
    for i in test_idx:
        X_test = X_test.append(sequences[i], ignore_index = True)    
    return X_train, X_train.iloc[:, -1], X_test, X_test.iloc[:, -1]
        
def find_perm(clusters, Y_real, Y_pred):
    perm = []
    for i in range(clusters):
        idx = Y_pred == i
        new_label = mode(Y_real[idx])[0][0]
        perm.append(new_label)
    return [perm[int(label)] for label in Y_pred]

def get_score(y_test, y_pred, T):
    score = np.count_nonzero(y_test == y_pred)
    print(f"Percentage of correct states: {np.round(score / len(y_test) * 100, 3)}%")
    
    score = 0
    for i in range(int(len(y_test) / T)):
        if np.all(y_test.iloc[i*T:(i+1)*T] == y_pred[i*T:(i+1)*T]):
            score += 1
    score /= len(y_test) / T
    print(f"Percentage of correct paths: {np.round(score * 100, 3)}%")
    
    lengths = []
    for i in range(int(len(y_test) / T)):
        is_equal = y_test.iloc[i*T:(i+1)*T] == y_pred[i*T:(i+1)*T]
        ilength = 0
        max_length = 0
        for j in is_equal:
            if j:
                ilength += 1
            else:
                max_length = max([max_length, ilength])
                ilength = 0
        max_length = max([max_length, ilength])
        lengths.append(max_length)
    print(f"Average correct subsequence length: {int(np.mean(lengths))}")

In [4]:
T = 12
while True:
    X_train, y_train, X_test, y_test = train_test_split(df, T)
    if len(np.unique(y_test)) == len(condition_dictionary):
        break

In [5]:
unique_conditions = condition_dictionary.values()
condition_product = [list(iprod) for iprod in product(unique_conditions, unique_conditions)]

initial_conditions = y_train.iloc[0:len(y_train):12]
pi = np.zeros(len(unique_conditions))
A = np.zeros(len(condition_product)) 

for i, ci in enumerate(initial_conditions):
    pi[ci] += 1
    split_idx_begin = i * T
    split_idx_end = (i+1) * T
    conditions = y_train.iloc[split_idx_begin:split_idx_end]
    conditions.reset_index(drop=True)
    condition_sequences = [[cond, conditions.iloc[icond + 1]] for icond, cond in enumerate(conditions[:-1])]
    for iseq in condition_sequences:
        for j, jprod in enumerate(condition_product):
            if iseq == jprod:
                A[j] += 1
                break

pi /= initial_conditions.shape
A = A.reshape(len(unique_conditions), len(unique_conditions))
A = A / A.sum(axis=0, keepdims=True)[:, np.newaxis]
A = np.squeeze(A, axis=0)
# print(np.allclose(np.sum(A, axis=0), 1))
print(f"pi = {pi} \nA = \n{A}")

pi = [0.60240964 0.25301205 0.12048193 0.02409639 0.        ] 
A = 
[[0.94117647 0.07719298 0.125      0.04878049 0.        ]
 [0.04117647 0.87017544 0.04166667 0.19512195 0.4       ]
 [0.01568627 0.01754386 0.80555556 0.07317073 0.        ]
 [0.00196078 0.03508772 0.02777778 0.58536585 0.6       ]
 [0.         0.         0.         0.09756098 0.        ]]


In [6]:
model = hmm.GaussianHMM(n_components=len(unique_conditions), covariance_type="full")
model.fit(X_train)
y_pred = model.predict(X_test)
y_pred = find_perm(len(np.unique(y_test)), y_test, y_pred)
get_score(y_test, y_pred, T)

Percentage of correct states: 68.056%
Percentage of correct paths: 36.111%
Average correct subsequence length: 7


In [9]:
model = hmm.GaussianHMM(n_components=len(unique_conditions), covariance_type="full", init_params="cm", params="cmt") 
model.startprob_ = pi
model.transmat_ = A.T
model.fit(X_train)
y_pred = model.predict(X_test)
y_pred = find_perm(len(np.unique(y_test)), y_test, y_pred)
get_score(y_test, y_pred, T)

Percentage of correct states: 92.593%
Percentage of correct paths: 72.222%
Average correct subsequence length: 10
